# Coloumb Matrix Elements

\begin{align*}
\langle 12| V | 34 \rangle =& 
\delta_{s_1, s_4} \delta_{s_2, s_3} \delta_{m_1 + m_2, m_3 + m_4}
\Big[\prod_{i=1}^4 \frac{n_i!}{(|m_i| + n_i)} \Big]^{1/2}
\sum_{(4)j=0}^n \frac{(-1)^{j_1+j_2+j_3+j_4}}{j_1!j_2!j_3!j_4!} \\
&\times \Big[\prod_{i=1}^n {n_i + |m_i| \choose n_i + j_i} \Big]
\frac{1}{2^{(G+1)/2}} \sum_{(4)l=0}^\gamma (-1)^{\gamma_2 + \gamma_3 - l_2 - l_3} \\
&\times \delta_{l_1 + l_2, l_3 + l_4} \Big[\prod_{i=1}^4 {\gamma_i \choose l_i} \Big] \Gamma\Big(1 + \frac{\Lambda}{2} \Big)\Gamma\Big(\frac{G - \Lambda + 1}{2}\Big)
\end{align*}

The symbols $j_i$ are integer summation indices (regular indices) running from $0$ to $n_i$.
The symbols $\gamma_i$ stand for numbers,
\begin{align*}
\gamma_1 &= j_1 + j_4 + (|m_1| + m_1)/2 + (|m_4| - m_4)/2 \\
\gamma_4 &= j_1 + j_4 + (|m_1| - m_1)/2 + (|m_4| + m_4)/2
\end{align*}
$\gamma_2$ and $\gamma_3$ can be obtained by replacing indices $1 \to 2$ and $4 \to 3$.

Moreover,
\begin{align*}
\sum_{(4)j=0}^n =
\sum_{j_1=0}^{n_1}\sum_{j_2=0}^{n_2}\sum_{j_3=0}^{n_3}\sum_{j_4=0}^{n_4},
\quad 
G = \sum_i \gamma_i, 
\quad
\Lambda = \sum_i l_i
\end{align*}

In [14]:
import numpy as np
import numba
from scipy.special import factorial, comb, gamma

In [2]:
delta = lambda p, q: p == q

In [15]:
@numba.njit
def _compute_prefactor(n, m):
    return np.sqrt(np.prod(factorial(n)/factorial(np.abs(m) + n)))

In [17]:
@numba.njit
def _compute_l_sum(gam):
    l_sum = 0
    G = np.sum(gam)
    l = np.zeros_like(gam)
    for l_1 in range(gam[0]):
        for l_2 in range(gam[1]):
            for l_3 in range(gam[2]):
                for l_4 in range(gam[3]):
                    if l_1 + l_2 != l_3 + l_4:
                        continue
                    sign = (-1)**(gam[1] + gam[2] - l_2 - l_3)

                    l[0] = l_1
                    l[1] = l_2
                    l[2] = l_3
                    l[3] = l_4
                    #l = np.array([l_1, l_2, l_3, l_4])
                    LAMBDA = np.sum(l)

                    term = np.prod(comb(gam, l))
                    term *= gamma(1 + LAMBDA/2.0)
                    term *= gamma((G - LAMBDA + 1)/2.0)

                    l_sum += sign * term

    return l_sum

In [21]:
@numba.njit
def compute_two_body_element(_1, _2, _3, _4):
    n_1, m_1, spin_1 = _1
    n_2, m_2 , spin_2 = _2
    n_3, m_3, spin_3 = _3
    n_4, m_4, spin_4 = _4

    n = np.zeros(4)
    m = np.zeros_like(n)
    j = np.zeros_like(n)
    gam = np.zeros_like(n)

    n[0] = n_1
    n[1] = n_2
    n[2] = n_3
    n[3] = n_4

    m[0] = m_1
    m[1] = m_2
    m[2] = m_3
    m[3] = m_4

    #n = np.array([n_1, n_2, n_3, n_4])
    #m = np.array([m_1, m_2, m_3, m_4])

    if spin_1 != spin_4 or spin_2 != spin_3:
        return 0

    if m_1 + m_2 != m_3 + m_4:
        return 0

    prefactor = _compute_prefactor(n, m)
    element = 0

    for j_1 in range(n_1):
        fact_1 = factorial(j_1)
        for j_2 in range(n_2):
            fact_2 = factorial(j_2)
            for j_3 in range(n_3):
                fact_3 = factorial(j_3)
                for j_4 in range(n_4):
                    fact_4 = factorial(j_4)

                    #j = np.array([j_1, j_2, j_3, j_4])
                    j[0] = j_1
                    j[1] = j_2
                    j[2] = j_3
                    j[3] = j_4
                    
                    gam[0] = j_1 + j_4 + (abs(m_1) + m_1)/2 + (abs(m_4) - m_4)/2
                    gam[1] = j_2 + j_3 + (abs(m_2) + m_2)/2 + (abs(m_3) - m_3)/2
                    gam[2] = j_3 + j_2 + (abs(m_3) + m_3)/2 + (abs(m_2) - m_2)/2
                    gam[3] = j_4 + j_1 + (abs(m_4) + m_4)/2 + (abs(m_1) - m_1)/2
                    #gam = np.array([gam_1, gam_2, gam_3, gam_4])

                    divisor = fact_1 * fact_2 * fact_3 * fact_4
                    sign = -1 if (j_1 + j_2 + j_3 + j_4) & 0x1 != 0 else 1
                    term_1 = sign / float(divisor)

                    G = np.sum(gam)
                    term_2 = np.prod(comb(n + np.abs(m), n - j)) / 2**((G + 1)/2.0)

                    l_sum = _compute_l_sum(gam)

                    element += term_1 * term_2 * l_sum

    return element

In [22]:
_0 = (0, 0, 0)
print (compute_two_body_element(_0, _0, _0, _0))

TypingError: Failed at nopython (nopython frontend)
Untyped global name 'factorial': cannot determine Numba type of <class 'function'>
File "<ipython-input-21-5ee2089ac761>", line 36